In [4]:
#goals in this project
#set up a black scholes pricing model
#calculate greeks based on black scholes

#come up with several other pricing methods too
    #binomial
    #monte carlo
    #heston

#test them against each other
#test their sensititives
#create unit tests

#test with market data

#find discerpencies
#explain discrepencies
#find way to exploit discrepencies

In [5]:
%run blackscholes.ipynb

In [6]:
#testing prices, verified these are accurate with an online calculator
testEC = BSCall(100,110,5,.05,.1)
testEC.print()

testEP = BSPut(100,110,5,.05,.1)
testEP.print()

The option has the following outputs: 
Price: $17.3198
Delta: $0.7892
Gamma: $0.0129
Vega: $64.5904
Theta: $-3.7258
Rho: $307.9937

The option has the following outputs: 
Price: $2.9879
Delta: $-0.2108
Gamma: $0.0129
Vega: $64.5904
Theta: $0.5576
Rho: $-120.3467

